<a href="https://colab.research.google.com/github/Sonal-Shriya-Pai/Quantum/blob/main/PulseViewerSol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compile quantum circuits into pulses
In this first exercise, you will get an idea of unitary gate implementation through direct visualization of pulses.

In this notebook, you will ...
* Prepare a GHZ state and compile the corresponding quantum circuit into a *playlist* of microwave pulses.

By the end of this notebook, you will understand the correspondence between unitary gates and their physical implementation for superconducting qubits.

In order to get started, make sure you have the appropriate packages installed:

In [ ]:
!pip install pip==23.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
# %%capture
!pip install iqm-client==32.1.1
!pip install iqm-station-control-client==11.3.1
!pip install iqm-pulla==11.16.2
!pip install iqm-pulse==12.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.4/148.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.5/99.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.1/421.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.0/544.0 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 560.9/560.9 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=15864c9610c05a0c8db87b167ea06e6734a501c5306dc96ffbb45ee60ff32e24
  Stored in directory: /root/.cache/pip/wheels/b1/7a/33/9fdd892f784ed4afda62b685ae3703adf4c91aa0f524c28f03
Successfully built pylatexenc
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.8/417.8 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: iqm-pulse
    Found existing installation: iqm-pulse 12.7.1
    Uninstalling iqm-pulse-12.7.1:
      Successfully uninstalled iqm-pulse-12.7.1


In [ ]:
!pip install qrisp[iqm]

## 1. Preparation

### 1.1 Connecting to the QPU station control
In the following, we will use the PulLa (Pulse Level Access) package. You can find the documentation [here](https://docs.meetiqm.com/iqm-pulla/).

As a first step, we need to create a **PulLa object**. In general, this is a **compiler**, in a particular state, linked to a particular quantum computer. It contains calibration data and the set of available operations.

Make sure you have the correct url and token below.

In [ ]:
from iqm.pulla.pulla import Pulla

api_token = input()
p = Pulla("https://cocos.resonance.meetiqm.com/garnet", get_token_callback=lambda: api_token)

compiler = p.get_standard_compiler()

## 2. GHZ state

You have learnt in the past labs and exercises what a GHZ state is and how to prepare one on a quantum computer. Here, we are not interested in its entanglement properties, but we will use it as a starting point to look into physical pulses. Let's construct a GHZ for 3 qubits:

In [ ]:
from qrisp import QuantumCircuit

qc_0 = QuantumCircuit(3)
qc_0.h(0)
qc_0.cx(0, 1)
qc_0.cx(0, 2)
qc_0.measure([0,1,2])

print(qc_0)

Before compiling the circuit, we need to transpile it to take into account the native gates and connectivity of IQM quantum computers:

In [ ]:
from iqm.qiskit_iqm import IQMProvider

provider = IQMProvider('https://cocos.resonance.meetiqm.com/garnet', token=api_token)
backend = provider.get_backend()

transpiled_qc = qc_0.transpile(backend = backend)
print(transpiled_qc)

To use the PulLA visualization tool, we need to convert the qrisp circuit into a qiskit one. We use the converter as below:

In [ ]:
qiskit_circuit = transpiled_qc.to_qiskit()

## 3. Compilation

We can now extract the list of *instructions* from the circuit defined above and compile it to obtain a *playlist* of pulses!

In [ ]:
from iqm.qiskit_iqm.qiskit_to_iqm import serialize_instructions
from iqm.iqm_client import Circuit

iqm_instructions = serialize_instructions(qiskit_circuit, {i : "QB" + str(i+1) for i in range(qiskit_circuit.num_qubits)})
iqm_circuit = Circuit(name = "my_circuit", instructions = iqm_instructions)

playlist, context = compiler.compile([iqm_circuit])

Finally, we are ready to visualize the pulses that prepare a GHZ state of 3 qubits. Let's explore the details:

In [ ]:
from iqm.pulse.playlist.visualisation.base import inspect_playlist
from IPython.core.display import HTML

HTML(inspect_playlist(playlist))